In [ ]:
import random

class Carta:
    def __init__(self, nombre, velocidad, fuerza, elemento, peso, altura):
        self.nombre = nombre
        self.velocidad = velocidad
        self.fuerza = fuerza
        self.elemento = elemento  # 1: agua, 2: tierra, 3: aire, 4: fuego, 5: electricidad
        self.peso = peso
        self.altura = altura

class Nodo:
    def __init__(self, carta=None, siguiente=None):
        self.carta = carta
        self.siguiente = siguiente

pila_uno = None
pila_dos = None
pila_mesa = None

cantidad_cartas_jugador_uno = 0
cantidad_cartas_jugador_dos = 0

nombre_jugador_uno = input("Ingrese nombre del jugador 1: ")
nombre_jugador_dos = input("Ingrese nombre del jugador 2 o deje en blanco si quiere jugar contra la pc: ")
if not nombre_jugador_dos:
    nombre_jugador_dos = "PC"

cartas = [None] * 250

def leer_cartas():
    with open("cartas.txt", "r") as file:
        for i in range(250):
            datos = file.readline().strip().split(',')
            cartas[i] = Carta(
                datos[0],
                int(datos[1]),
                int(datos[2]),
                int(datos[3]),
                float(datos[4]),
                float(datos[5])
            )

def agregar_carta(carta, nodo_superior):
    nuevo_nodo = Nodo(carta, nodo_superior)
    return nuevo_nodo

def agregar_carta_final(nodo_superior):
    pila_auxiliar = None
    contador_cartas_nuevas = 0

    while nodo_superior:
        carta = sacar_carta(nodo_superior)
        pila_auxiliar = agregar_carta(carta, pila_auxiliar)

    while pila_mesa:
        carta = sacar_carta(pila_mesa)
        nodo_superior = agregar_carta(carta, nodo_superior)
        contador_cartas_nuevas += 1

    while pila_auxiliar:
        carta = sacar_carta(pila_auxiliar)
        nodo_superior = agregar_carta(carta, nodo_superior)

    return contador_cartas_nuevas

def sacar_carta(nodo_superior):
    if nodo_superior:
        carta_actual = nodo_superior.carta
        nodo_superior = nodo_superior.siguiente
        return carta_actual
    return None

def mezclar_cartas():
    for i in range(250):
        posicion_aleatoria = random.randint(0, 249)
        cartas[i], cartas[posicion_aleatoria] = cartas[posicion_aleatoria], cartas[i]

# Cargar las cartas en las pilas de los jugadores
leer_cartas()
mezclar_cartas()

for i in range(125):
    pila_uno = agregar_carta(cartas[i], pila_uno)
    cantidad_cartas_jugador_uno += 1

for i in range(125, 250):
    pila_dos = agregar_carta(cartas[i], pila_dos)
    cantidad_cartas_jugador_dos += 1

maximo_rondas = 250
ronda_actual = 1

while pila_uno and pila_dos and ronda_actual <= maximo_rondas:
    atributo_sorteado = random.randint(1, 5)
    carta_jugador_uno = sacar_carta(pila_uno)
    carta_jugador_dos = sacar_carta(pila_dos)
    cantidad_cartas_jugador_uno -= 1
    cantidad_cartas_jugador_dos -= 1

    atributo_valor_uno = getattr(carta_jugador_uno, list(vars(carta_jugador_uno).keys())[atributo_sorteado])
    atributo_valor_dos = getattr(carta_jugador_dos, list(vars(carta_jugador_dos).keys())[atributo_sorteado])

    if atributo_valor_uno > atributo_valor_dos:
        pila_mesa = agregar_carta(carta_jugador_uno, pila_mesa)
        pila_mesa = agregar_carta(carta_jugador_dos, pila_mesa)
        cantidad_cartas_jugador_uno += agregar_carta_final(pila_uno)
        print("Esta mano la ganó el jugador uno")
    
    elif atributo_valor_uno < atributo_valor_dos:
        pila_mesa = agregar_carta(carta_jugador_uno, pila_mesa)
        pila_mesa = agregar_carta(carta_jugador_dos, pila_mesa)
        cantidad_cartas_jugador_dos += agregar_carta_final(pila_dos)
        print("Esta mano la ganó el jugador dos")
    
    else:
        pila_mesa = agregar_carta(carta_jugador_uno, pila_mesa)
        pila_mesa = agregar_carta(carta_jugador_dos, pila_mesa)
        print("Esta mano fue empate")

    ronda_actual += 1

if ronda_actual == maximo_rondas:
    print("Juego finalizado por límite de rondas")

if cantidad_cartas_jugador_uno > cantidad_cartas_jugador_dos:
    print(f"Ganó jugador: {nombre_jugador_uno}")
elif cantidad_cartas_jugador_uno < cantidad_cartas_jugador_dos:
    if nombre_jugador_dos != "PC":
        print(f"Ganó jugador: {nombre_jugador_dos}")
    else:
        print("Ganó la maquina")
else:
    print("Empate")
